In [ ]:
!pip install duckduckgo-search

/Users/I756498/.asdf/installs/python/3.13.1/lib/python3.13/pty.py:95: DeprecationWarning: This process (pid=6867) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
zsh:1: no matches found: smolagents[litellm]


/Users/I756498/.asdf/installs/python/3.13.1/lib/python3.13/pty.py:95: DeprecationWarning: This process (pid=6867) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


In [19]:
import os
import requests
import re
from datetime import datetime, timedelta
from smolagents import CodeAgent, DuckDuckGoSearchTool, LiteLLMModel, tool, Tool

# Weather Tool
OPENWEATHER_API_KEY = "bd5e378503939ddaee76f12ad7a97608"
SPOONACULAR_API_KEY = "f2f6b1736b0e488d9bf536af6b017519"
REMINDERS_API_KEY = "vCulKBidqDF0MwUza6GSL5kTWwpj66hb6rWmGxYS"
BASE_URL = "https://api.spoonacular.com"

categories = ['Weather', 'Fitness', 'Wellbeing', 'Sleep', 'Calendar', 'Nutrition', 'Reminder', 'Search', 'Unknown']

# Categories and descriptions
category_descriptions = {
    "Weather": "query include weather-related intent",
    "Fitness": "query asks about exercise or fitness routines",
    "Wellbeing": "query shares emotional, mood, or journal-related inputs but not general questions",
    "Sleep": "query talks about sleep, fatigue, or rest",
    "Calendar": "query refers to appointments or events",
    "Nutrition": "query include meal names",
    "Reminder": "query include intent with reminding something",
    "Search": "query include a question but it is not related to the other categories",#  as long as it is not related to weather, fitness, wellbeing, sleep, calender, nutrition, nor reminder",
}

# Build the classification system prompt
desc_lines = "\n".join(f"{cat}: {desc}" for cat, desc in category_descriptions.items())
prompt_router = f"""
You are a classification assistant. Your ONLY task is to analyze user's paragraph then split it into parts then categorize them based on the categories provided using their description.
CATEGORIES:
{desc_lines}

Instructions:
1) Read the full paragraph, analyze it and split it all into parts.
2) For each part, decide which category (or multiple) it belongs to based on intent and your understanding from the predefined CATEGORIES making sure they fit the descriptions provided.
3) If a part does not fit any of the predefined CATEGORIES, then put it as UNKNOWN category.
4) DO NOT provide answers, suggestions, extra commentary, or explanations.
5) DO NOT return anything except classified part under each category name as the OUTPUT FORMAT.
6) Make sure all the input text was split properly and classified, so nothing is missing from the input

OUTPUT FORMAT:
category_name:
- [Sentence].
"""

@tool
def get_weather(data: dict, ctx: dict) -> dict:
    """
    Fetch current weather from OpenWeatherMap and update context.

    Args:
        data (dict): Contains 'location' with 'lat' and 'lon'.
        ctx (dict): User context to update.

    Returns:
        dict: Weather info including 'city', 'temp', 'condition', and 'wind_kph'.
    """
    loc = data.get('location', {})
    lat = loc.get('lat')
    lon = loc.get('lon')
    if lat is None or lon is None:
        return {"error": "Location not provided"}
    key = os.getenv("OPENWEATHER_API_KEY", OPENWEATHER_API_KEY)
    try:
        r = requests.get(
            "https://api.openweathermap.org/data/2.5/weather",
            params={"lat": lat, "lon": lon, "units": "metric", "appid": key},
            timeout=5
        ).json()
        info = {
            "city": r.get("name", ""),
            "temp": r["main"]["temp"],
            "condition": r["weather"][0]["description"],
            "wind_kph": round(r["wind"]["speed"] * 3.6, 1)
        }
    except Exception as e:
        return {"error": str(e)}
    ctx['weather'] = info
    return info


class SpoonacularTool(Tool):
    print("INVOKING")
    inputs = {
        "input_text": {
            "type": "string",
            "description": (
                "Command: 'search:<query>' or 'details:<id>'"
            )
        }
    }
    output_type = "string"
    name = "SpoonacularTool"
    description = (
        "Use this tool to search recipes or fetch full details: instructions, nutrition, taste"
    )

    def forward(self, input_text: str) -> str:
        print("INPUT_TEXT", input_text)
        command, _, arg = input_text.partition(":")
        if command == "search":
            return self.search_recipes(arg)
        if command == "details":
            return self.get_nutritional_info(arg)
        return f"Unknown command: {input_text}"

    def search_recipes(self, query: str, number: int = 1) -> str:
        url = f"{BASE_URL}/recipes/complexSearch"
        params = {
            'query': query,
            'number': number,
            'addRecipeInformation': False,
            'apiKey': SPOONACULAR_API_KEY
        }
        resp = requests.get(url, params=params)
        resp.raise_for_status()
        results = resp.json().get('results', [])
        return "\n".join(f"{r['id']}: {r['title']}" for r in results)

    def get_nutritional_info(self, recipe_id: str) -> str:
        nut = requests.get(
            f"{BASE_URL}/recipes/{recipe_id}/nutritionWidget.json",
            params={'apiKey': SPOONACULAR_API_KEY}
        ).json()
        # Assemble output
        out = "**Instructions:**\n"
        out += "\n**Nutrition per serving:**\n"
        out += f"- Calories: {nut['calories']}\n"
        out += f"- Carbs: {nut['carbs']}\n"
        out += f"- Fat: {nut['fat']}\n"
        out += f"- Protein: {nut['protein']}\n\n"
        out += "**Taste Profile:**\n"
        return out

class ReminderTool(Tool):
    inputs = {
        "reminder_text": {
            "type": "string",
            "description": "Reminder content to schedule"
        }
    }
    output_type = "string"
    name = "ReminderTool"
    description = "Tool to create a new reminder by sending it to the reminders API."

    def forward(self, reminder_text: str) -> str:
        print("REMINDER TEXT", reminder_text)
        url = "https://reminders-api.com/api/applications/1231/reminders/"
        headers = {
            "Authorization": f"Bearer {REMINDERS_API_KEY}",
            "Content-Type": "application/json"
        }
        title = reminder_text.strip().capitalize()
        timezone = "Europe/Paris"
        now = datetime.now()
        # extract date
        date_match = re.search(r"\b(\d{4}-\d{2}-\d{2})\b", reminder_text)
        if date_match:
            date_str = date_match.group(1)
        else:
            date_str = (now + timedelta(days=1)).strftime("%Y-%m-%d")

        # extract time
        time_match = re.search(r"\b(\d{2}:\d{2})\b", reminder_text)
        if time_match:
            time_str = time_match.group(1)
        else:
            time_str = "09:00"
        
        notify_match = re.search(r"\b(in|before)\s+(\d+\s+(minutes?|hours?|days?|weeks?))", reminder_text, re.IGNORECASE)
        notify_in_advance = notify_match.group(2) if notify_match else "10 minutes"
        payload = {
            "title": title,
            "timezone": timezone,
            "date_tz": date_str,
            "time_tz": time_str,
            "notify_in_advance": notify_in_advance,
            "snoozed": 0
        }
        print("payload: ", payload)
        try:
            response = requests.post(url, headers=headers, json=payload)
            response.raise_for_status()
            return f"Reminder created for {date_str} at {time_str}: {title}"
        except Exception as e:
            return f"Failed to create reminder: {e}"


model = LiteLLMModel(model_id="ollama/qwen2.5-coder:3b")

def generate_prompt(name, purpose, capabilities, returns, final_only=True):
    parts = [
        f"You are {name}.",
        f"Purpose: {purpose}",
        "Capabilities:",
        *[f"- {c.strip()}" for c in capabilities.split(';') if c.strip()],
        "Returns:",
        *[f"- {r.strip()}" for r in returns.split(';') if r.strip()],
        "Only output the final result in plain text."
    ]
    if final_only:
        parts.append("Do not show any code, plan, or explanation.")
    parts.append("If you cannot respond, return 'NO_RESPONSE'.")
    return "\n".join(parts)

# Define prompts
prompt_wellbeing = generate_prompt(
    "Wellbeing Companion",
    "Provide psychological support and mental wellness tips based on user journal or mood inputs.",
    "Analyze emotional tone; Suggest wellness activities; Support stress and anxiety",
    "Short-term coping tips; Actionable wellness advice"
)

prompt_fitness = generate_prompt(
    "Fitness Coach",
    "Provides tailored workout recommendations based on user requests and environmental factors to have a complete workout or help the user with his fitness requests.",
    "Use weather and activity history; Recommend suitable fitness activities and help the user with his requests",
    "A personalized activity suggestion"
)

prompt_search = generate_prompt(
    "Search Assistant",
    "Search for answers to user queries when others cannot respond.",
    "Perform web search using DuckDuckGoSearchTool; Summarize key result",
    "Concise factual answer"
)

prompt_nutrition = (
    """
    IMPORTANT: This is a food-related query. Please provide comprehensive nutritional information including:
    - Calories per serving
    - Protein, carbohydrates, and fat content
    - Key vitamins and minerals
    - Health benefits and dietary considerations
    - Suggestions for healthier ingredient substitutions when applicable
    - Use the search tool to find accurate nutritional data from reliable sources like USDA food database

    Present the nutritional information prominently in your response.
    """
)

prompt_reminder = {
    """
IMPORTANT: This is a reminder-related query. You must create and schedule a reminder using the provided content. Extract and structure the information accurately.

You must determine and prepare the following fields:
- Title: A clear, concise description of what the reminder is for.
- Date (date_tz): The local date when the reminder should trigger. Use YYYY-MM-DD format. If not specified, default to tomorrow.
- Time (time_tz): The local time when the reminder should trigger. Use HH:MM format. If not specified, default to 09:00.
- Timezone: Use "Europe/Helsinki" unless another timezone is clearly specified.
- Notify in Advance (notify_in_advance): How much earlier the user should be notified. Accept formats like "10 minutes", "1 hour", or "2 days". If unspecified, default to "10 minutes".
- Optional Recurrence (rrule): Include only if user mentions repeating, recurring, or scheduling something regularly.
- Optional Snoozed (snoozed): Set to 1 if user explicitly says to pause, snooze, or delay the reminder.

Use this information to call the ReminderTool with the correct structure.

Only return the final confirmation message:
- Confirm the reminder was scheduled
- Mention the title, date, time, and advance notification

Do not show code or tool calls. Do not explain your process. If unable to create the reminder, return "NO_RESPONSE".

"""
}


# Initialize agents
agent_wellbeing = CodeAgent(model=model, tools=[], max_steps=2, verbosity_level=-1)
agent_fitness = CodeAgent(model=model, tools=[get_weather], max_steps=2, verbosity_level=-1)
agent_search = CodeAgent(model=model, tools=[DuckDuckGoSearchTool()], max_steps=2, verbosity_level=-1)
agent_nutrition = CodeAgent(model=model, tools=[SpoonacularTool(), DuckDuckGoSearchTool()], max_steps=2, verbosity_level=-1)
agent_reminder = CodeAgent(model=model, tools=[ReminderTool()], max_steps=2, verbosity_level=-1)
# agent_nutrition.prompt_templates['system_prompt'] = prompt_nutrition


agent_router = CodeAgent(model=model, tools=[], max_steps=3, verbosity_level=-1)

# Output dictionary setup
agent_prompt = {cat: [] for cat in categories}
agent_prompt["Wellbeing"] = [agent_wellbeing, prompt_wellbeing]
agent_prompt["Fitness"] = [agent_fitness, prompt_fitness]
agent_prompt["Search"] = [agent_search, prompt_search]
agent_prompt["Nutrition"] = [agent_nutrition, prompt_nutrition]
agent_prompt["Reminder"] = [agent_reminder, prompt_reminder]


def router_helper(raw_outputs):
    print("RAW OUTPUTS: \n", raw_outputs)
    # Categories list
    categories = ['Weather', 'Fitness', 'Wellbeing', 'Sleep', 'Calendar', 'Nutrition', 'Reminder', 'Search', 'Unknown']

    # Output dictionary setup
    output_dict = {cat: [] for cat in categories}
    # Extract each category block


    # Extract each category block
    blocks = re.split(r'\n(?=[A-Za-z]+:)', raw_outputs.strip())

    for block in blocks:
        match = re.match(r'([A-Za-z]+):\s*(.*)', block, re.DOTALL)
        if not match:
            continue
        category, content = match.groups()
        category = category.strip().capitalize()

        if category not in output_dict:
            continue

        # Extract both bracketed sentences and those starting with "-"
        bracket_sentences = re.findall(r'\[([^\[\]]+?)\]', content)
        dash_sentences = re.findall(r'-\s*([^\n]+)', content)

        # Combine and clean
        all_sentences = bracket_sentences + dash_sentences

        for sentence in all_sentences:
            sentence = sentence.replace(']',"").replace('[',"").strip().rstrip('.').lower()
            if sentence not in output_dict[category]:
                output_dict[category].append(sentence)


    combined_output = {k: ', & '.join(v) for k, v in output_dict.items()}
    print("combined_output OUTPUTS: ", combined_output)
    return combined_output

def agent_invoker(input_text, weather_text, prompt_type, agent_type, curr_agent):
    enriched = f"{prompt_type}\nWeather: {weather_text}\nUser: {input_text}"
    print(f"\n[Invoking {agent_type} Agent]")
    response = curr_agent.run(enriched).strip()
    if response != 'NO_RESPONSE':
        pass
    print(f"\n[{agent_type} Agent Response]", response)
    return response

# Router
def router(user_input: str, location: dict) -> str:
    ctx = {}

    print("\n--- ROUTER INPUT ---")
    print(f"Input: {user_input}")

    # Weather
    weather_info = get_weather({"location": location}, ctx)
    if "error" not in weather_info:
        weather_text = (
            f"Current weather in {weather_info['city']}: {weather_info['temp']}°C, "
            f"{weather_info['condition']}"
        )
    else:
        weather_text = "Weather info unavailable"
    print("\n[Weather Context]", weather_text)

    full_prompt = prompt_router + "\nUser Input: " + user_input
    
    # Segment and classify
    combined_output = router_helper(agent_router.run(full_prompt))
    all_responses = []
    for k, v in combined_output.items():
        # Compose and run wellbeing agent
        if v:
            if k in ["Wellbeing", "Fitness", "Search"]:
                response = agent_invoker(v, weather_text, agent_prompt[k][1], k, agent_prompt[k][0])
            elif k == ["Nutrition"]:
                print(f"\n[Invoking Nutrition Agent]")
                response = agent_nutrition.run(v)
                print(f"\n[Nutrition Agent Response]", response)
            elif k == "Reminder":
                print(f"\n[Invoking Reminder Agent]")
                response = agent_reminder.run(v)
                print(f"\n[Reminder Agent Response]", response)
            else: # Default to search
                response = agent_invoker(v, weather_text, prompt_search, "Search", agent_search)
            
            all_responses.append(response)
            

# --- Test ---
if __name__ == '__main__':
    test_input = "I am feeling energetic today, I want to do a good workout that includes going" \
    " to the swimming pool due to it being very hot today," \
    " also add to my training biking and" \
    " tell me a good place to do it. I need to eat to boost my protein intake and remind me to drink water in 1 hour"
    location = {'lat': 48.8566, 'lon': 2.3522}  # Paris
    output = router(test_input, location)
    # print(output)

INVOKING

--- ROUTER INPUT ---
Input: I am feeling energetic today, I want to do a good workout that includes going to the swimming pool due to it being very hot today, also add to my training biking and tell me a good place to do it. I need to eat to boost my protein intake and remind me to drink water in 1 hour

[Weather Context] Current weather in Paris: 18.75°C, overcast clouds


/Users/I756498/.asdf/installs/python/3.13.1/lib/python3.13/site-packages/smolagents/default_tools.py:135: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  self.ddgs = DDGS(**kwargs)
/Users/I756498/.asdf/installs/python/3.13.1/lib/python3.13/site-packages/smolagents/default_tools.py:135: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  self.ddgs = DDGS(**kwargs)
/Users/I756498/.asdf/installs/python/3.13.1/lib/python3.13/site-packages/httpx/_models.py:408: DeprecationWarning: Use 'content=<...>' to upload raw bytes/text content.
  headers, stream = encode_request(
/Users/I756498/.asdf/installs/python/3.13.1/lib/python3.13/site-packages/httpx/_models.py:408: DeprecationWarning: Use 'content=<...>' to upload raw bytes/text content.
  headers, stream = encode_request(
/Users/I756498/.asdf/installs/python/3.13.1/lib/python3.13/site-packages/httpx/_models.py:408: Dep

RAW OUTPUTS: 
 Weather:
- I want to do a good workout that includes going to the swimming pool due to it being very hot today.

Fitness:
- I want to do a good workout, biking.
- Tell me a good place to do it.

Wellbeing:
- I am feeling energetic today.

Nutrition:
- I need to eat to boost my protein intake.

Reminder:
- Remind me to drink water in 1 hour.
combined_output OUTPUTS:  {'Weather': 'i want to do a good workout that includes going to the swimming pool due to it being very hot today', 'Fitness': 'i want to do a good workout, biking, & tell me a good place to do it', 'Wellbeing': 'i am feeling energetic today', 'Sleep': '', 'Calendar': '', 'Nutrition': 'i need to eat to boost my protein intake', 'Reminder': 'remind me to drink water in 1 hour', 'Search': '', 'Unknown': ''}

[Invoking Search Agent]


/Users/I756498/.asdf/installs/python/3.13.1/lib/python3.13/site-packages/httpx/_models.py:408: DeprecationWarning: Use 'content=<...>' to upload raw bytes/text content.
  headers, stream = encode_request(
/Users/I756498/.asdf/installs/python/3.13.1/lib/python3.13/site-packages/httpx/_models.py:408: DeprecationWarning: Use 'content=<...>' to upload raw bytes/text content.
  headers, stream = encode_request(



[Search Agent Response] Swimming is an excellent exercise for burning calories and improving cardiovascular health when the temperature is high. Some recommended activities include:
- **Free泳 (Freestyle)**: This technique involves using all four limbs while moving through water.
- **蛙泳 (Bikini)**: A popular style that allows you to stay afloat with your legs.
- **仰泳 (Breaststroke)**: Uses the arms and chest for propulsion, which is less strenuous than freestyle on the back.
- **蝶泳 (Butterfly)**: This involves both arms and legs moving simultaneously in a circular motion.

Consider choosing an activity that suits your comfort level and fitness goals. Always warm up before swimming to prevent injuries.

[Invoking Fitness Agent]


...


[Fitness Agent Response] **Bike Rental Locations in Paris**

Paris is known for its iconic bike lanes and public transportation system, making it an ideal location for a good cycling workout. Here are some popular bike rental locations:

1. **Cathédrale Notre-Dame de la Garde**: This cathedral offers extensive paths for biking, including the Grand Louvre Metro stop.

2. **Jardin du Luxembourg**: The park provides ample space for a leisurely or more energetic cycle ride around its winding paths and gardens.

3. **Bercy Riverfront Park**: Located near the riverbank, this area offers scenic views of the Seine while providing good cycling opportunities.

4. **Montmartre**: This historic neighborhood has many bicycle rental shops, particularly around the square near Sacré-Cœur Basilica.

5. **Gare de Lyon (Saint-Germain-des-Halles)**: This bustling metro station has several bike racks available for public use.

6. **Parc Montsouris**: A large park with several cycling paths and a lake, ide

/Users/I756498/.asdf/installs/python/3.13.1/lib/python3.13/site-packages/httpx/_models.py:408: DeprecationWarning: Use 'content=<...>' to upload raw bytes/text content.
  headers, stream = encode_request(
/Users/I756498/.asdf/installs/python/3.13.1/lib/python3.13/site-packages/httpx/_models.py:408: DeprecationWarning: Use 'content=<...>' to upload raw bytes/text content.
  headers, stream = encode_request(
/Users/I756498/.asdf/installs/python/3.13.1/lib/python3.13/site-packages/httpx/_models.py:408: DeprecationWarning: Use 'content=<...>' to upload raw bytes/text content.
  headers, stream = encode_request(



[Wellbeing Agent Response] **Wellbeing Companion:** 
- **Short-term coping tips:** Continue engaging in activities that bring you joy and relaxation.
- **Actionable wellness advice:** Focus on maintaining a consistent routine to support your energy levels and well-being.

[Invoking Search Agent]


/Users/I756498/.asdf/installs/python/3.13.1/lib/python3.13/site-packages/httpx/_models.py:408: DeprecationWarning: Use 'content=<...>' to upload raw bytes/text content.
  headers, stream = encode_request(
/Users/I756498/.asdf/installs/python/3.13.1/lib/python3.13/site-packages/httpx/_models.py:408: DeprecationWarning: Use 'content=<...>' to upload raw bytes/text content.
  headers, stream = encode_request(
/Users/I756498/.asdf/installs/python/3.13.1/lib/python3.13/site-packages/httpx/_models.py:408: DeprecationWarning: Use 'content=<...>' to upload raw bytes/text content.
  headers, stream = encode_request(
/Users/I756498/.asdf/installs/python/3.13.1/lib/python3.13/site-packages/httpx/_models.py:408: DeprecationWarning: Use 'content=<...>' to upload raw bytes/text content.
  headers, stream = encode_request(



[Search Agent Response] To boost your protein intake, consider incorporating lean sources of protein like eggs, chicken breast, fish (such as salmon or tuna), lentils, chickpeas, and beans into your meals. These options are low in fat and calories while providing essential amino acids needed for muscle repair and growth. Additionally, aim to consume a balanced diet rich in vegetables, fruits, whole grains, and healthy fats to ensure you meet all your nutritional needs.

[Invoking Reminder Agent]
REMINDER TEXT Drink water!
payload:  {'title': 'Drink water!', 'timezone': 'Europe/Paris', 'date_tz': '2025-07-16', 'time_tz': '09:00', 'notify_in_advance': '10 minutes', 'snoozed': 0}

[Reminder Agent Response] Reminder created
